# Fred

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
import datetime

In [2]:
orders=pd.read_csv('QuoteItemLabelData.csv')

In [3]:
orders=orders[orders['brand_name']=='louis vuitton']

In [4]:
orders['submitted_date']=pd.to_datetime(orders['submitted_date'])
orders['style_name']=orders['style_name'].str.lower()

In [5]:
count=orders.groupby(['style_name', 'style_size_name', 'print_name'])['total'].count().reset_index()
count

,style_name,style_size_name,print_name,total
0,abbesses,none,monogram,1
1,agenda,mm,epi,1
2,agenda,mm,monogram,1
3,agenda,pm,damier azur,7
4,agenda,pm,damier ebene,7
...,...,...,...,...
460,zippy wallet,none,giant monogram,7
461,zippy wallet,none,monogram,182
462,zippy wallet,none,monogram multicolor black,5
463,zippy wallet,none,monogram multicolor white,6


In [6]:
count['style_name']=count['style_name'].str.lower()

In [7]:
fred=pd.read_excel('fred_item.xlsx')

In [8]:
fred['item_name']=fred['item_name'].str.lower()

In [9]:
new=fred["item_name"].str.split(" ", expand = True)
new=new.rename(columns={0:'style_name', 1: 'style_size_name'})
new['style_name']=new['style_name'].str.lstrip()

In [10]:
carry=[]
for s in list(count['style_name']):
    for i in list(new['style_name']):
        if i in s:
            carry.append(s)

new_count=count[count['style_name'].isin(carry)]

In [11]:
# count.to_excel('item_count.xlsx')

In [12]:
orders.columns

Index(['item_id', 'brand_name', 'style_name', 'style_size_name',
       'material_name', 'material_type', 'print_name', 'title', 'color',
       'category', 'original_price', 'overall_grade', 'external_grade',
       'internal_grade', 'buyer', 'seller', 'total', 'submitted_date',
       'margin', 'auction_flag'],
      dtype='object')

In [13]:
recent=orders[orders['submitted_date']>datetime.datetime(2021,1,1)]

In [14]:
price=recent.groupby(['style_name', 'style_size_name', 'print_name','overall_grade'])['total'].mean().rename('avg_price').reset_index()
price=price.pivot_table(values=['avg_price'],
                        index=['style_name', 'style_size_name', 'print_name'], columns=['overall_grade'])

In [15]:
updated=new_count.merge(price, left_on=['style_name', 'style_size_name', 'print_name'], right_index=True, how='left')

/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [16]:
updated

,style_name,style_size_name,print_name,total,"(avg_price, A)","(avg_price, AB)","(avg_price, B)","(avg_price, BC)","(avg_price, C)","(avg_price, CD)","(avg_price, NS)","(avg_price, S)","(avg_price, SA)"
9,alize 2,none,monogram,3,751.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,alma,bb,damier ebene,3,1226.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,alma,bb,epi,21,1319.0,1043.400000,949.600000,NaN,NaN,NaN,NaN,NaN,NaN
12,alma,bb,monogram,49,1048.4,996.230769,1079.000000,NaN,1095.0,NaN,NaN,NaN,NaN
13,alma,gm,epi,4,661.0,670.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,zippy wallet,none,giant monogram,7,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,1091.0,NaN
461,zippy wallet,none,monogram,182,508.0,452.454545,473.402778,513.0,NaN,NaN,NaN,NaN,NaN
462,zippy wallet,none,monogram multicolor black,5,NaN,528.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
463,zippy wallet,none,monogram multicolor white,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# updated.to_excel('item_count.xlsx')